# 엔티티만 변경, 다른 부분은 고정!

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [117]:
# 데이터 안에서 원하는 tpye의 entity 리스트 추출
    # PER, LOC, NOH, ORG, DAT, 
def extract_entity_based_on_type(data, entity, type_, save_name):
    et = {'name':[], 'type':[]}
    for dic in data[entity].values:
        dic = literal_eval(dic)

        if dic['type'] == type_:
            et['name'].append(dic['word'])

    names = list(set(et['name']))
    et['name'] = names
    et['type'] = [type_ for _ in range(len(names))]
    et = pd.DataFrame(et)
    #et.to_csv(save_name+'.csv', index=False)
    
    return et

In [226]:
# Sentence 변경
def change_sentence(text, sx, ex, sy, ey, first_word, second_word):
    # entity 위치만 제거 하고 새로운 entity 삽입
    sentence = text[:sx] + first_word + text[ex+1:sy] + second_word + text[ey+1:]
    # 새로운 좌표계산
    origin_length = ex - sx + 1 
    ex = sx + len(first_word) - 1  # 길이를 이용해 끝점 계산
    sy = sy + (len(first_word) - origin_length) # 길면 +, 짧으면 - 해서 시작위치 조절
    ey = sy + len(second_word) - 1 # 길이를 이용해 끝점 계산
    
    return sentence, sx, ex, sy, ey

In [223]:
def augment_data(target_df, sub_entity_list, obj_entity_list, target_class, times, save_name):
    new = {'id':[], 'sentence':[], 'subject_entity':[], 'object_entity':[], 'label':[], 'source':[]}
    
    # 원본 데이터수 * times 개 생성
    for time in range(times):
        for idx, (id_, sentence, subject_entity, object_entity, label, source) in enumerate(target_df.values):
            # 정보 추출
            subject_dict = literal_eval(subject_entity)
            object_dict = literal_eval(object_entity)

            # 랜덤 추출
            sub_idx = np.random.randint(0, sub_entity_list.shape[0])
            obj_idx = np.random.randint(0, obj_entity_list.shape[0])
            sub_word = sub_entity_list['name'].iloc[sub_idx]
            obj_word = obj_entity_list['name'].iloc[obj_idx]

            # Sentence 바꾸기
                # 먼저 나오는 entity 속성에 따라 입력 값, 반환 값 순서를 다르게!
            if subject_dict['start_idx'] > object_dict['start_idx']:
                new_sentence, sy, ey, sx, ex = change_sentence(sentence, object_dict['start_idx'], object_dict['end_idx'], subject_dict['start_idx'], subject_dict['end_idx'], obj_word, sub_word)
            else:
                new_sentence, sx, ex, sy, ey = change_sentence(sentence, subject_dict['start_idx'], subject_dict['end_idx'], object_dict['start_idx'], object_dict['end_idx'], sub_word, obj_word)

            # subject_entity//object_entity 새로운 정보 입력
            subject_dict['word'] = sub_word
            subject_dict['start_idx'] = sx
            subject_dict['end_idx'] = ex

            object_dict['word'] = obj_word
            object_dict['start_idx'] = sy
            object_dict['end_idx'] = ey

            new['id'].append(id_)
            new['sentence'].append(new_sentence)
            new['subject_entity'].append(str(subject_dict)) # string으로 변환해 입력
            new['object_entity'].append(str(object_dict)) # string으로 변환해 입력
            new['label'].append(label)
            new['source'].append(source) 

    # 데이터 프레임으로 변경
    new = pd.DataFrame(new)
    # 문장 기준 중복제거
    new = new.drop_duplicates('sentence')
    print("생성된 데이터 수:", new.shape)
    #new.to_csv('./entity_change_data/new_'+target_class+'_members.csv', index=False)
    new.to_csv('./entity_change_data/' + save_name , index=False)
    
    return new

In [120]:
# [[40, 'per:place_of_death'], 2000대
#  [48, 'org:number_of_employees/members'], 2000대
#  [66, 'org:dissolved'], 2000대
#  [82, 'per:schools_attended'], 2000대
#  [96, 'per:religion'], (2196, 6)
#  [98, 'org:political/religious_affiliation'], (2296, 6)
#  [136, 'per:siblings'], (2040, 6)
#  [139, 'per:product'], (2224, 6)
#  [155, 'org:founded_by'], (2015, 6)
#  [166, 'per:place_of_birth'], (2158, 6)
#  [190, 'per:other_family'], (2090, 6)
#  [193, 'per:place_of_residence'], (2123, 6)
#  [304, 'per:children'], (2128, 6)
#  [380, 'org:product'], (2280, 6)
#  [418, 'per:date_of_death'], (2090, 6)
#  [420, 'org:members'], (1260, 6)
#  [450, 'org:founded'], (2250, 6)
#  [520, 'per:parents'], (2080, 6)
#  [534, 'per:colleagues'], (2136, 6)
#  [795, 'per:spouse'], (2385, 6)
#  [1001, 'per:alternate_names'], (1001, 6)
#  [1130, 'per:date_of_birth'],
#  [1195, 'org:place_of_headquarters'],
#  [1234, 'per:origin'],
#  [1320, 'org:alternate_names'],
#  [1866, 'org:member_of'],
#  [2103, 'per:title'],
#  [3573, 'per:employee_of'],
#  [4284, 'org:top_members/employees'],
#  [9534, 'no_relation']]

In [121]:
data = pd.read_csv('./train/train.csv')
print(data.shape)

(32470, 6)


In [227]:
orgs = pd.read_csv('./object_ORG_list.csv') # or extract_entity_based_on_type(data, 'subject_entity' ,'ORG')
dats = pd.read_csv('./object_DAT_list.csv') # or extract_entity_based_on_type(data, 'object_entity' ,'DAT')
pers = pd.read_csv('./object_PER_list.csv') # or extract_entity_based_on_type(data, 'subject_entity' ,'PER')
locs = pd.read_csv('./object_LOC_list.csv') # or extract_entity_based_on_type(data, 'subject_entity' ,'LOC')
nohs = pd.read_csv('./object_NOH_list.csv') # or extract_entity_based_on_type(data, 'object_entity' ,'NOH')
pros= pd.read_csv('./object_product_list.csv.csv')
religion = pd.read_csv('./object_Religion_list.csv.csv')
school = pd.read_csv('./object_school_list.csv.csv')

In [220]:
target_label = 'per:place_of_death'

In [221]:
target_df = data.loc[data.label == target_label]
print(target_df.shape)

(40, 6)


In [187]:
pohs = extract_entity_based_on_type(target_df, 'object_entity', 'POH', './object_orgproduct_list.csv')

In [222]:
new_df = augment_data(target_df, pers, dats, target_label, 50)

생성된 데이터 수: (2000, 6)


In [246]:
target_label = 'per:place_of_death'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, dats, target_label, 2500//len(target_df), f'new_{target_label}.csv')

target_label = 'org:number_of_employees/members'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, nohs, target_label, 2500//len(target_df), f'new_org:number_of_employees_members.csv')

target_label = 'org:dissolved'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, dats, target_label, 2500//len(target_df), f'new_{target_label}.csv')

target_label = 'per:schools_attended'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, school, target_label, 2500//len(target_df), f'new_{target_label}.csv')

target_label = 'per:religion'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, religion, target_label, 2500//len(target_df), f'new_{target_label}.csv')

target_label = 'org:political/religious_affiliation'
target_df = data.loc[data.label == target_label]
pohs = extract_entity_based_on_type(target_df, 'object_entity', 'POH', './religious_affiliation_list.csv')
new_df = augment_data(target_df, orgs, pohs, target_label, 2500//len(target_df), f'new_org:political_religious_affiliation')

target_label = 'per:siblings'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:product'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pros, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'org:founded_by'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, orgs, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:place_of_birth'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, locs, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:other_family'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:place_of_residence'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, locs, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:children'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'org:product'
target_df = data.loc[data.label == target_label]
pohs = extract_entity_based_on_type(target_df, 'object_entity', 'POH', './org:product_list.csv')
new_df = augment_data(target_df, orgs, pohs, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:date_of_death'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, dats, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'org:members'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, orgs, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'org:founded'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, dats, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:parents'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 2000//len(target_df), f'new_{target_label}.csv')

target_label = 'per:colleagues'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 1500//len(target_df), f'new_{target_label}.csv')

target_label = 'per:spouse'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 1500//len(target_df), f'new_{target_label}.csv')

target_label = 'per:alternate_names'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, pers, target_label, 1500//len(target_df), f'new_{target_label}.csv')

target_label = 'per:date_of_birth'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, dats, target_label,1500//len(target_df), f'new_{target_label}.csv')

target_label = 'org:place_of_headquarters'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, locs, target_label, 1500//len(target_df), f'new_{target_label}.csv')

target_label = 'per:origin'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, pers, locs, target_label, 1500//len(target_df), f'new_{target_label}.csv')

target_label = 'org:alternate_names'
target_df = data.loc[data.label == target_label]
new_df = augment_data(target_df, orgs, orgs, target_label, 1500//len(target_df), f'new_{target_label}.csv')

# target_label = 'org:member_of'
# target_df = data.loc[data.label == target_label]
# new_df = augment_data(target_df, orgs, orgs, target_label, 1500//len(target_df), f'new_{target_label}.csv')

# target_label = 'per:title'
# target_df = data.loc[data.label == target_label]
# pohs = extract_entity_based_on_type(target_df, 'object_entity', 'POH', './titles.csv')
# new_df = augment_data(target_df, pers, pohs, target_label, 1500//len(target_df), f'new_{target_label}.csv')

생성된 데이터 수: (2480, 6)
생성된 데이터 수: (2496, 6)
생성된 데이터 수: (2442, 6)
생성된 데이터 수: (2460, 6)
생성된 데이터 수: (2496, 6)
생성된 데이터 수: (2450, 6)
생성된 데이터 수: (1904, 6)
생성된 데이터 수: (1946, 6)
생성된 데이터 수: (1860, 6)
생성된 데이터 수: (1992, 6)
생성된 데이터 수: (1900, 6)
생성된 데이터 수: (1930, 6)
생성된 데이터 수: (1824, 6)
생성된 데이터 수: (1900, 6)
생성된 데이터 수: (1672, 6)
생성된 데이터 수: (1680, 6)
생성된 데이터 수: (1800, 6)
생성된 데이터 수: (1560, 6)
생성된 데이터 수: (1068, 6)
생성된 데이터 수: (795, 6)
생성된 데이터 수: (1001, 6)
생성된 데이터 수: (1130, 6)
생성된 데이터 수: (1195, 6)
생성된 데이터 수: (1234, 6)
생성된 데이터 수: (1320, 6)


In [88]:
pers

,name,type,length
0,앨리슨 슈밋,PER,6
1,강명도,PER,3
2,윤종신,PER,3
3,미유키,PER,3
4,테드 크루즈,PER,6
...,...,...,...
7399,모리에 히로시,PER,7
7400,남창희,PER,3
7401,딥플로우,PER,4
7402,오준혁,PER,3
